<a href="https://colab.research.google.com/github/KshKnsl/MinorProject/blob/main/ollama.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!curl -fsSL https://ollama.com/install.sh | sh

>>> Installing ollama to /usr/local
>>> Downloading Linux amd64 bundle
######################################################################## 100.0%
>>> Creating ollama user...
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.


In [ ]:
!pip install aiohttp pyngrok

import os
import asyncio
import aiohttp

os.environ.update({'LD_LIBRARY_PATH': '/usr/lib64-nvidia'})

async def run_process(cmd):
  print('>>> starting', *cmd)
  p = await asyncio.subprocess.create_subprocess_exec(
      *cmd,
      stdout=asyncio.subprocess.PIPE,
      stderr=asyncio.subprocess.PIPE,
  )

  async def pipe(lines):
    async for line in lines:
      print(line.strip().decode('utf-8'))

  await asyncio.gather(
      pipe(p.stdout),
      pipe(p.stderr),
  )
await asyncio.gather(
    run_process(['ngrok', 'config', 'add-authtoken','32eB7tLSQoICKJD4JSQuJ9lWea6_7U5ndjtQCVaWnPLEc4Mws'])
)

ngrok_task_for_url = asyncio.create_task(run_process(['ngrok', 'http', '--log', 'stderr', '11434', '--host-header', 'localhost:11434']))

await asyncio.sleep(5)

try:
    async with aiohttp.ClientSession() as session:
        async with session.get('http://localhost:4040/api/tunnels') as resp:
            data = await resp.json()
            ngrok_url = None
            for tunnel in data['tunnels']:
                if tunnel['proto'] == 'https':
                    ngrok_url = tunnel['public_url']
                    break
            if ngrok_url:
                print(f"\nNgrok Public URL: {ngrok_url}\n")
                os.environ['OLLAMA_ORIGINS'] = ngrok_url
            else:
                print("\nCould not find an HTTPS ngrok tunnel.\n")

except Exception as e:
    print(f"\nCould not retrieve ngrok public URL: {e}\n")
    ngrok_url = None

ollama_task = asyncio.create_task(run_process(['ollama', 'serve']))

await asyncio.gather(ollama_task, ngrok_task_for_url)

>>> starting ngrok config add-authtoken 32eB7tLSQoICKJD4JSQuJ9lWea6_7U5ndjtQCVaWnPLEc4Mws
Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml
>>> starting ollama serve
>>> starting ngrok http --log stderr 11434 --host-header localhost:11434
time=2025-09-14T14:35:02.183Z level=INFO source=routes.go:1331 msg="server config" env="map[CUDA_VISIBLE_DEVICES: GPU_DEVICE_ORDINAL: HIP_VISIBLE_DEVICES: HSA_OVERRIDE_GFX_VERSION: HTTPS_PROXY: HTTP_PROXY: NO_PROXY: OLLAMA_CONTEXT_LENGTH:4096 OLLAMA_DEBUG:INFO OLLAMA_FLASH_ATTENTION:false OLLAMA_GPU_OVERHEAD:0 OLLAMA_HOST:http://127.0.0.1:11434 OLLAMA_INTEL_GPU:false OLLAMA_KEEP_ALIVE:5m0s OLLAMA_KV_CACHE_TYPE: OLLAMA_LLM_LIBRARY: OLLAMA_LOAD_TIMEOUT:5m0s OLLAMA_MAX_LOADED_MODELS:0 OLLAMA_MAX_QUEUE:512 OLLAMA_MODELS:/root/.ollama/models OLLAMA_MULTIUSER_CACHE:false OLLAMA_NEW_ENGINE:false OLLAMA_NEW_ESTIMATES:false OLLAMA_NOHISTORY:false OLLAMA_NOPRUNE:false OLLAMA_NUM_PARALLEL:1 OLLAMA_ORIGINS:[http://localhost https://localhost ht

Ngrok exposes a url, which you the have to export as OLLAMA_HOST

`export OLLAMA_HOST=https://b028594345f0.ngrok-free.app/`

after that we can use ollama on our remote instance from our local machine.

In [ ]:
!pip install flask flask-cors